# Huggingface AK and research community papers

Flow to get info from AK and research community papers

## Initial setup

In [1]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from scripts.hf_scraper_script import HFPapersScraper
from scripts.arxiv_utils import *

## HuggingFace paper parsing

In [2]:
# initialize Huggingface scraper class
hf_papers_scraper = HFPapersScraper()

# get all the papers from selected date (or latest date - default page)
hf_links = hf_papers_scraper.get_paper_urls()

# get arxiv ids from the links
arxiv_ids = hf_papers_scraper.get_paper_ids()

## Getting a list of authors from the papers

In [3]:
# Loop through the arxiv ids and get the authors through the arxiv API and XML parsing
for arxiv_id in arxiv_ids:
    xml_content = get_content_by_id(arxiv_id)
    print(arxiv_id, extract_paper_info(xml_content)['authors'])

2411.04335 ['He-Yen Hsieh', 'Ziyun Li', 'Sai Qian Zhang', 'Wei-Te Mark Ting', 'Kao-Den Chang', 'Barbara De Salvo', 'Chiao Liu', 'H. T. Kung']
2411.05005 ['Shuhong Zheng', 'Zhipeng Bao', 'Ruoyu Zhao', 'Martial Hebert', 'Yu-Xiong Wang']
2411.05003 ['David Junhao Zhang', 'Roni Paiss', 'Shiran Zada', 'Nikhil Karnad', 'David E. Jacobs', 'Yael Pritch', 'Inbar Mosseri', 'Mike Zheng Shou', 'Neal Wadhwa', 'Nataniel Ruiz']
2411.04905 ['Siming Huang', 'Tianhao Cheng', 'Jason Klein Liu', 'Jiaran Hao', 'Liuyihan Song', 'Yang Xu', 'J. Yang', 'J. H. Liu', 'Chenchen Zhang', 'Linzheng Chai', 'Ruifeng Yuan', 'Zhaoxiang Zhang', 'Jie Fu', 'Qian Liu', 'Ge Zhang', 'Zili Wang', 'Yuan Qi', 'Yinghui Xu', 'Wei Chu']
2411.04965 ['Hongyu Wang', 'Shuming Ma', 'Furu Wei']
2411.04989 ['Koichi Namekata', 'Sherwin Bahmani', 'Ziyi Wu', 'Yash Kant', 'Igor Gilitschenski', 'David B. Lindell']
2411.04923 ['Shehan Munasinghe', 'Hanan Gani', 'Wenqi Zhu', 'Jiale Cao', 'Eric Xing', 'Fahad Shahbaz Khan', 'Salman Khan']
2411.044